### Q1. He 초기화를 사용하여 무작위로 선택한 값이라면 모든 가중치를 같은 값으로 초기화해도 괜찮을까?

(답변) 안 된다. 애초에 초기화를 하는 이유는 가중치마다 서로 다른 임의의 값을 갖게 하기 위함이기 때문이다. 

### Q2. 편향을 0으로 초기화해도 괜찮을까?

(답변) 편향은 초기화하지 않아도 무관하기 때문에 상관 없다. 합연산이기 때문에 ?

### Q3. ReLU보다 ELU 활성화 함수가 나은 세 가지는 무엇일까요?

(답변) 음의 값이 나와도 미분이 가능하다. 학습 속도가 빠르다. 유연하다.

### Q4. 어떤 경우에 ELU, LeakyReLU, ReLU, tanh, logistic, softmax와 같은 활성화 함수를 사용해야 할까?

(답변) 출력이 분류인 문제일 경우에 사용한다.

### Q5. MomentumOptimizer를 사용할 때 momentum 하이퍼파라미터를 너무 1에 가깝게 하면 어떤 일이 일어날까?

(답변) 발산한다.

### Q6. 희소 모델을 만들 수 있는 세 가지 방법은?

(답변) 모른다.

### Q7. 드롭아웃이 훈련 속도를 느리게 만들까? 추론(새로운 샘플에 대한 예측을 만드는 것)도 느리게 만들까?

(답변) 뉴런의 수가 줄어들기 때문에 더 빨라진다.

### Q8. 딥러닝 실습

- a. He 초기화와 ELU 활성화 함수를 사용하여 각각 뉴런이 100개인 은닉층 다섯 개를 가진 DNN을 만드세요.
- b. Adam 최적화와 조기 종료를 사용하여 MNIST 데이터셋에 훈련시키되 0에서 4까지의 숫자만 사용하세요. 다음 연습문제에서 5에서 9까지의 숫자에 대해 전이 학습을 사용할 것입니다. 출력층은 다섯 개의 뉴런에 소프트맥스 함수를 사용하고, 나중에 재사용할 수 있도록 항상 일정한 간격으로 체크포인트와 최종 모델을 저장하세요.
- c. 교차 검증을 사용하여 하이퍼파라미터를 튜닝하고 얼마의 성능을 달성할 수 있는지 확인해보세요.
- d. 배치 정규화를 추가한 다음 학습 곡선을 비교해보세요. 이전보다 수렴이 빨라졌나요? 모델의 성능이 더 나아졌나요?
- e. 모델이 훈련 세트에 과대적합되었나요? 모든 층에 드롭아웃을 적용하고 다시 시도해보세요. 도움이 되었나요

### a. 기본 모델

In [1]:
import tensorflow as tf
import numpy as np
import datetime

C:\Users\ud803\Anaconda3\envs\hands_on_ml\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ud803\Anaconda3\envs\hands_on_ml\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ud803\Anaconda3\envs\hands_on_ml\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ud803\Anaconda3\envs\hands_on_ml\lib\site-pack

In [2]:
# Prepare Datasets
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_train = X_train[y_train <= 4]
X_test = X_test[y_test <= 4]
y_train = y_train[y_train <= 4]
y_test = y_test[y_test <= 4]
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

# Prepare Functions
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [3]:
n_inputs = 28*28 #MNIST
n_hidden1, n_hidden2, n_hidden3, n_hidden4, n_hidden5 = [100] * 5
n_outputs = 5

In [6]:
tf.reset_default_graph()

# Tensorflow Preparation
he_init = tf.variance_scaling_initializer()

# Tensorflow Graph Declaration
## X, Y Var
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

## DNN
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.elu, kernel_initializer=he_init)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.elu, kernel_initializer=he_init)
    hidden3 = tf.layers.dense(hidden2, n_hidden3, name="hidden3", activation=tf.nn.elu, kernel_initializer=he_init)
    hidden4 = tf.layers.dense(hidden3, n_hidden4, name="hidden4", activation=tf.nn.elu, kernel_initializer=he_init)
    hidden5 = tf.layers.dense(hidden4, n_hidden5, name="hidden5", activation=tf.nn.elu, kernel_initializer=he_init)
    logits = tf.layers.dense(hidden5, n_outputs, name="logits", kernel_initializer=he_init)
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

## LOSS
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

## Training
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

## Eval
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")    


In [7]:
## Execution
init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 40
batch_size = 50

best_loss = np.inf
max_try = 20
current_try = 0

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        save_path = saver.save(sess, "models/tmp/chap11_exercise.ckpt")

        
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        loss_valid = loss.eval(feed_dict={X: X_valid, y: y_valid})
        
        if acc_valid < best_loss:
            save_path = saver.save(sess, "models/chap11_exercise.ckpt")
            best_loss = loss_valid
            current_try = 0
        else:
            current_try += 1
            if current_try >= max_try:
                print("조기 종료!")
                break
        print(epoch, "검증 세트 손실:", loss_valid, "최선의 손실:", best_loss, "검증 세트 정확도:", acc_valid)

0 검증 세트 손실: 0.1143729 최선의 손실: 0.1143729 검증 세트 정확도: 0.9684
1 검증 세트 손실: 0.7052579 최선의 손실: 0.1143729 검증 세트 정확도: 0.9286
2 검증 세트 손실: 0.08815428 최선의 손실: 0.1143729 검증 세트 정확도: 0.9738
3 검증 세트 손실: 0.14425915 최선의 손실: 0.1143729 검증 세트 정확도: 0.9634
4 검증 세트 손실: 0.12995398 최선의 손실: 0.1143729 검증 세트 정확도: 0.9656
5 검증 세트 손실: 0.05894281 최선의 손실: 0.1143729 검증 세트 정확도: 0.9868
6 검증 세트 손실: 0.07724715 최선의 손실: 0.1143729 검증 세트 정확도: 0.981
7 검증 세트 손실: 0.08882479 최선의 손실: 0.1143729 검증 세트 정확도: 0.9746
8 검증 세트 손실: 0.08154777 최선의 손실: 0.1143729 검증 세트 정확도: 0.9844
9 검증 세트 손실: 0.1851009 최선의 손실: 0.1143729 검증 세트 정확도: 0.9802
10 검증 세트 손실: 0.09481877 최선의 손실: 0.1143729 검증 세트 정확도: 0.9746
11 검증 세트 손실: 0.11775906 최선의 손실: 0.1143729 검증 세트 정확도: 0.976
12 검증 세트 손실: 0.09208188 최선의 손실: 0.1143729 검증 세트 정확도: 0.9812
13 검증 세트 손실: 0.09289459 최선의 손실: 0.1143729 검증 세트 정확도: 0.9822
14 검증 세트 손실: 0.082883 최선의 손실: 0.1143729 검증 세트 정확도: 0.9844
15 검증 세트 손실: 0.07850165 최선의 손실: 0.1143729 검증 세트 정확도: 0.9858
16 검증 세트 손실: 0.27883676 최선의 손실: 0.1143729 검증 세트 정확도: 0.97

In [6]:
with tf.Session() as sess:
    saver.restore(sess, "models/chap11_exercise.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
    print("최종 테스트 정확도: {:.2f}%".format(acc_test * 100))

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from models/chap11_exercise.ckpt
최종 테스트 정확도: 97.94%


### b. 교차 검증

In [36]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [37]:
X_train1 = X_train[y_train < 5]
y_train1 = y_train[y_train < 5]
X_valid1 = X_valid[y_valid < 5]
y_valid1 = y_valid[y_valid < 5]
X_test1 = X_test[y_test < 5]
y_test1 = y_test[y_test < 5]

In [7]:
import sklearn

sklearn.__version__

'0.22'

In [56]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.exceptions import NotFittedError

class DNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden_layers = 5, n_neurons = 100, optimizer_class = tf.train.AdamOptimizer, learning_rate = 0.01, 
                 batch_size = 20, activation = tf.nn.elu, initializer = he_init, batch_norm_momentum = None, dropout_rate = None, random_staet = None ):
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.optimizer_class = optimizer_class
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.activation = activation
        self.initializer = initializer
        self.batch_norm_momentum = batch_norm_momentum
        self.dropout_rate = dropout_rate
        self._session = None

    def _dnn(self, inputs):
        for layer in range(self.n_hidden_layers):
            if self.dropout_rate:
                inputs = tf.layers.dropout(inputs, self.dropout_rate, training=self._training)
            inputs = tf.layers.dense(inputs, self.n_neurons, kernel_initializer=self.initializer, name="hidden%d" % (layer + 1))

            if self.batch_norm_momentum:
                inputs = tf.layers.batch_normalization(inputs, momentum=self.batch_norm_momentum, training=self._training)
            inputs = self.activation(inputs, name="hidden%d_out" % (layer + 1))
        return inputs
        
    def _close_session(self):
        if self._session:
            self._session.close()
            
    def _build_graph(self, n_inputs, n_outputs):
        # Tensorflow Graph Declaration
        ## X, Y Var
        X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
        y = tf.placeholder(tf.int32, shape=(None), name="y")

        if self.batch_norm_momentum or self.dropout_rate:
            self._training = tf.placeholder_with_default(False, shape=(), name='training')
        else:
            self._training = None
            
        ## DNN
        dnn_outputs = self._dnn(X)
        logits = tf.layers.dense(dnn_outputs, n_outputs, name="logits", kernel_initializer=he_init)
        Y_proba = tf.nn.softmax(logits, name="Y_proba")
        
        ## LOSS
        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")

        ## Training
        optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
        training_op = optimizer.minimize(loss)

        ## Eval
        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))    
        
        ## Init & Save
        init = tf.global_variables_initializer()
        saver = tf.train.Saver()

        self._X, self._y = X, y
        self._Y_proba, self._loss = Y_proba, loss
        self._training_op, self._accuracy = training_op, accuracy
        self._init, self._saver = init, saver

    def _get_model_params(self):
        """모든 변수 값을 가져옵니다 (조기 종료를 위해 사용하며 디스크에 저장하는 것보다 빠릅니다)"""
        with self._graph.as_default():
            gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        return {gvar.op.name: value for gvar, value in zip(gvars, self._session.run(gvars))}
    
    def _restore_model_params(self, model_params):
        """모든 변수를 주어진 값으로 설정합니다 (조기 종료를 위해 사용하며 디스크에 저장하는 것보다 빠릅니다)"""
        gvar_names = list(model_params.keys())
        assign_ops = {gvar_name: self._graph.get_operation_by_name(gvar_name + "/Assign")
                      for gvar_name in gvar_names}
        init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
        feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
        self._session.run(assign_ops, feed_dict=feed_dict)
        
    def fit(self, X, y, n_epochs=1000, X_valid=None, y_valid=None):
        """훈련 세트에 모델을 훈련시킵니다. X_valid와 y_valid가 주어지면 조기 종료를 적용합니다."""
        self._close_session()

        # 훈련 세트로부터 n_inputs와 n_outputs를 구합니다.
        n_inputs = X.shape[1]
        self.classes_ = np.unique(y)
        n_outputs = len(self.classes_)
        
        # 레이블 벡터를 정렬된 클래스 인덱스 벡터로 변환합니다.
        # 0부터 n_outputs - 1까지의 정수를 담고 있게 됩니다.
        # 예를 들어, y가 [8, 8, 9, 5, 7, 6, 6, 6]이면 
        # 정렬된 클래스 레이블(self.classes_)은 [5, 6, 7, 8, 9]가 되고
        # 레이블 벡터는 [3, 3, 4, 0, 2, 1, 1, 1]로 변환됩니다.
        self.class_to_index_ = {label: index
                                for index, label in enumerate(self.classes_)}
        y = np.array([self.class_to_index_[label]
                      for label in y], dtype=np.int32)
        
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(n_inputs, n_outputs)
            # 배치 정규화를 위한 추가 연산
            extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

        # 조기 종료를 위해
        max_checks_without_progress = 20
        checks_without_progress = 0
        best_loss = np.infty
        best_params = None
        
        # 이제 모델을 훈련합니다!
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._init.run()
            for epoch in range(n_epochs):
                rnd_idx = np.random.permutation(len(X))
                for rnd_indices in np.array_split(rnd_idx, len(X) // self.batch_size):
                    X_batch, y_batch = X[rnd_indices], y[rnd_indices]
                    feed_dict = {self._X: X_batch, self._y: y_batch}
                    if self._training is not None:
                        feed_dict[self._training] = True
                    sess.run(self._training_op, feed_dict=feed_dict)
                    if extra_update_ops:
                        sess.run(extra_update_ops, feed_dict=feed_dict)
                if X_valid is not None and y_valid is not None:
                    loss_val, acc_val = sess.run([self._loss, self._accuracy],
                                                 feed_dict={self._X: X_valid,
                                                            self._y: y_valid})
                    if loss_val < best_loss:
                        best_params = self._get_model_params()
                        best_loss = loss_val
                        checks_without_progress = 0
                    else:
                        checks_without_progress += 1
                    print("{}\t검증 세트 손실: {:.6f}\t최선의 손실: {:.6f}\t정확도: {:.2f}%".format(
                        epoch, loss_val, best_loss, acc_val * 100))
                    if checks_without_progress > max_checks_without_progress:
                        print("조기 종료!")
                        break
                else:
                    loss_train, acc_train = sess.run([self._loss, self._accuracy],
                                                     feed_dict={self._X: X_batch,
                                                                self._y: y_batch})
                    print("{}\t마지막 훈련 배치 손실: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_train, acc_train * 100))
            # 조기 종료를 사용하면 이전의 최상의 모델로 되돌립니다.
            if best_params:
                self._restore_model_params(best_params)
            return self
        
    def predict_proba(self, X):
        if not self._session:
            raise NotFittedError("%s 객체가 아직 훈련되지 않았습니다" % self.__class__.__name__)
        with self._session.as_default() as sess:
            return self._Y_proba.eval(feed_dict={self._X: X})

    def predict(self, X):
        class_indices = np.argmax(self.predict_proba(X), axis=1)
        return np.array([[self.classes_[class_index]]
                         for class_index in class_indices], np.int32)

    def save(self, path):
        self._saver.save(self._session, path)

In [38]:
dnn_clf = DNNClassifier()
dnn_clf.fit(X_train1, y_train1, n_epochs=1000, X_valid=X_valid1, y_valid=y_valid1)

0	검증 세트 손실: 0.109910	최선의 손실: 0.109910	정확도: 97.62%
1	검증 세트 손실: 0.192316	최선의 손실: 0.109910	정확도: 95.23%
2	검증 세트 손실: 0.113300	최선의 손실: 0.109910	정확도: 97.54%
3	검증 세트 손실: 0.206293	최선의 손실: 0.109910	정확도: 95.35%
4	검증 세트 손실: 0.131957	최선의 손실: 0.109910	정확도: 97.65%
5	검증 세트 손실: 0.234273	최선의 손실: 0.109910	정확도: 95.78%
6	검증 세트 손실: 0.132155	최선의 손실: 0.109910	정확도: 97.46%
7	검증 세트 손실: 0.157653	최선의 손실: 0.109910	정확도: 96.64%
8	검증 세트 손실: 0.131527	최선의 손실: 0.109910	정확도: 97.42%
9	검증 세트 손실: 0.103936	최선의 손실: 0.103936	정확도: 97.97%
10	검증 세트 손실: 1.700996	최선의 손실: 0.103936	정확도: 18.73%
11	검증 세트 손실: 1.687720	최선의 손실: 0.103936	정확도: 22.01%
12	검증 세트 손실: 1.658944	최선의 손실: 0.103936	정확도: 22.01%
13	검증 세트 손실: 1.705560	최선의 손실: 0.103936	정확도: 18.73%
14	검증 세트 손실: 1.660976	최선의 손실: 0.103936	정확도: 19.27%
15	검증 세트 손실: 1.780466	최선의 손실: 0.103936	정확도: 19.27%
16	검증 세트 손실: 1.628658	최선의 손실: 0.103936	정확도: 22.01%
17	검증 세트 손실: 1.615816	최선의 손실: 0.103936	정확도: 22.01%
18	검증 세트 손실: 1.692735	최선의 손실: 0.103936	정확도: 19.08%
19	검증 세트 손실: 1.669257	최선의 손실: 0.103936	정확

C:\Users\ud803\Anaconda3\envs\hands_on_ml\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


DNNClassifier(activation=<function elu at 0x000002AD974CD048>,
              batch_norm_momentum=None, batch_size=20, dropout_rate=None,
              initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x000002AD9C5E88C8>,
              learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
              optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
              random_staet=None)

In [39]:
from sklearn.metrics import accuracy_score

y_pred = dnn_clf.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.9846273594084453

In [ ]:
rnd_search.best_params_

### Q9. 전이 학습
- a. 이전 모델에서 미리 학습한 은닉층을 모두 재사용하는 새로운 DNN을 만드세요. 이 은닉층을 동결시키고 소프트맥스 출력층은 새로운 것으로 바꾸세요.
- b. 이 새로운 DNN을 숫자 5~9에 대해 숫자마다 100개의 이미지만 사용해 훈련시켜 보고 얼마나 시간이 걸리는지 재보세요. 작은 양의 샘플만으로도 높은 성능을 얻을 수 있나요?
- c. 동결된 층을 캐싱하고 모델을 다시 훈련시켜보세요. 얼마나 빨라졌나요?
- d. 다섯 개 대신 네 개의 은닉층만 재사용하여 다시 시도해보세요. 높은 성능을 얻을 수 있나요?
- e. 이제 최상단 두 개 층의 동결을 해제하고 훈련을 계속 해보세요. 더 나은 모델을 얻을 수 있나요?

### a, b

In [10]:
import tensorflow as tf
import numpy as np

In [18]:
for op in tf.get_default_graph().get_operations():
    print(op.name)

X
y
hidden1/kernel/Initializer/truncated_normal/shape
hidden1/kernel/Initializer/truncated_normal/mean
hidden1/kernel/Initializer/truncated_normal/stddev
hidden1/kernel/Initializer/truncated_normal/TruncatedNormal
hidden1/kernel/Initializer/truncated_normal/mul
hidden1/kernel/Initializer/truncated_normal
hidden1/kernel
hidden1/kernel/Assign
hidden1/kernel/read
hidden1/bias/Initializer/zeros
hidden1/bias
hidden1/bias/Assign
hidden1/bias/read
dnn/hidden1/MatMul
dnn/hidden1/BiasAdd
dnn/hidden1/Elu
hidden2/kernel/Initializer/truncated_normal/shape
hidden2/kernel/Initializer/truncated_normal/mean
hidden2/kernel/Initializer/truncated_normal/stddev
hidden2/kernel/Initializer/truncated_normal/TruncatedNormal
hidden2/kernel/Initializer/truncated_normal/mul
hidden2/kernel/Initializer/truncated_normal
hidden2/kernel
hidden2/kernel/Assign
hidden2/kernel/read
hidden2/bias/Initializer/zeros
hidden2/bias
hidden2/bias/Assign
hidden2/bias/read
dnn/hidden2/MatMul
dnn/hidden2/BiasAdd
dnn/hidden2/Elu
hidd

In [11]:
tf.reset_default_graph()

restore_saver = tf.train.import_meta_graph("models/chap11_exercise.ckpt.meta")

X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")
loss = tf.get_default_graph().get_tensor_by_name("loss/loss:0")
Y_proba = tf.get_default_graph().get_tensor_by_name("dnn/Y_proba:0")
logits = Y_proba.op.inputs[0]
accuracy = tf.get_default_graph().get_tensor_by_name("eval/accuracy:0")

learning_rate = 0.01

output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam2")
training_op = optimizer.minimize(loss, var_list=output_layer_vars)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

init = tf.global_variables_initializer()
five_frozen_saver = tf.train.Saver()

In [12]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

X_train2_full = X_train[y_train >= 5]
y_train2_full = y_train[y_train >= 5] - 5
X_valid2_full = X_valid[y_valid >= 5]
y_valid2_full = y_valid[y_valid >= 5] - 5
X_test2 = X_test[y_test >= 5]
y_test2 = y_test[y_test >= 5] - 5

In [13]:
def sample_n_instances_per_class(X, y, n=100):
    Xs, ys = [], []
    for label in np.unique(y):
        idx = (y == label)
        Xc = X[idx][:n]
        yc = y[idx][:n]
        Xs.append(Xc)
        ys.append(yc)
    return np.concatenate(Xs), np.concatenate(ys)

In [14]:
X_train2, y_train2 = sample_n_instances_per_class(X_train2_full, y_train2_full, n=100)
X_valid2, y_valid2 = sample_n_instances_per_class(X_valid2_full, y_valid2_full, n=30)

In [32]:
import time

n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "models/chap11_exercise.ckpt")

    t0 = time.time()
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        
        if loss_val < best_loss:
            save_path = five_frozen_saver.save(sess, "models/chap11_exercise_five.ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("조기 종료!")
                break
        print("{}\t검증 세트 손실: {:.6f}\t최선의 손실: {:.6f}\t정확도: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

    t1 = time.time()
    print("전체 훈련 시간: {:.1f}s".format(t1 - t0))

with tf.Session() as sess:
    five_frozen_saver.restore(sess, "models/chap11_exercise_five.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("최종 테스트 정확도: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from models/chap11_exercise.ckpt
0	검증 세트 손실: 1.357602	최선의 손실: 1.357602	정확도: 40.00%
1	검증 세트 손실: 1.183432	최선의 손실: 1.183432	정확도: 58.67%
2	검증 세트 손실: 1.217803	최선의 손실: 1.183432	정확도: 53.33%
3	검증 세트 손실: 1.204619	최선의 손실: 1.183432	정확도: 56.00%
4	검증 세트 손실: 1.140937	최선의 손실: 1.140937	정확도: 56.67%
5	검증 세트 손실: 1.185161	최선의 손실: 1.140937	정확도: 55.33%
6	검증 세트 손실: 1.089114	최선의 손실: 1.089114	정확도: 60.00%
7	검증 세트 손실: 1.171013	최선의 손실: 1.089114	정확도: 58.00%
8	검증 세트 손실: 1.288682	최선의 손실: 1.089114	정확도: 50.00%
9	검증 세트 손실: 1.109311	최선의 손실: 1.089114	정확도: 57.33%
10	검증 세트 손실: 1.134755	최선의 손실: 1.089114	정확도: 56.00%
11	검증 세트 손실: 1.142793	최선의 손실: 1.089114	정확도: 54.00%
12	검증 세트 손실: 1.194862	최선의 손실: 1.089114	정확도: 52.00%
13	검증 세트 손실: 1.225149	최선의 손실: 1.089114	정확도: 52.00%
14	검증 세트 손실: 1.162608	최선의 손실: 1.089114	정확도: 56.00%
15	검증 세트 손실: 1.249606	최선의 손실: 1.089114	정확도: 55.33%
16	검증 세트 손실: 1.197919	최선의 손실: 1.089114	정확도: 56.67%
17	검증 세트 손실: 1.072532	최선의 손실: 1.072532	정확도: 64.00%
18	검증 세트 손실: 1.152226	

하나의 층만 사용했기 때문에 성능이 좋지는 않다.

### c.

In [28]:
hidden5_out = tf.get_default_graph().get_tensor_by_name("dnn/hidden5/Elu:0")

In [33]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "models/chap11_exercise.ckpt")

    t0 = time.time()
    
    hidden5_train = hidden5_out.eval(feed_dict={X: X_train2, y: y_train2})
    hidden5_valid = hidden5_out.eval(feed_dict={X: X_valid2, y: y_valid2})
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            h5_batch, y_batch = hidden5_train[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={hidden5_out: h5_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={hidden5_out: hidden5_valid, y: y_valid2})
        if loss_val < best_loss:
            save_path = five_frozen_saver.save(sess, "models/chap11_exercise_five.ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("조기 종료!")
                break
        print("{}\t검증 세트 손실: {:.6f}\t최선의 손실: {:.6f}\t정확도: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

    t1 = time.time()
    print("전체 훈련 시간: {:.1f}s".format(t1 - t0))

with tf.Session() as sess:
    five_frozen_saver.restore(sess, "models/chap11_exercise_five.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("최종 테스트 정확도: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from models/chap11_exercise.ckpt
0	검증 세트 손실: 1.219045	최선의 손실: 1.219045	정확도: 55.33%
1	검증 세트 손실: 1.168979	최선의 손실: 1.168979	정확도: 57.33%
2	검증 세트 손실: 1.220361	최선의 손실: 1.168979	정확도: 52.67%
3	검증 세트 손실: 1.134673	최선의 손실: 1.134673	정확도: 54.00%
4	검증 세트 손실: 1.127449	최선의 손실: 1.127449	정확도: 58.00%
5	검증 세트 손실: 1.141214	최선의 손실: 1.127449	정확도: 60.67%
6	검증 세트 손실: 1.176387	최선의 손실: 1.127449	정확도: 56.00%
7	검증 세트 손실: 1.181723	최선의 손실: 1.127449	정확도: 54.67%
8	검증 세트 손실: 1.189654	최선의 손실: 1.127449	정확도: 48.67%
9	검증 세트 손실: 1.082460	최선의 손실: 1.082460	정확도: 62.67%
10	검증 세트 손실: 1.144644	최선의 손실: 1.082460	정확도: 57.33%
11	검증 세트 손실: 1.217727	최선의 손실: 1.082460	정확도: 61.33%
12	검증 세트 손실: 1.135005	최선의 손실: 1.082460	정확도: 58.00%
13	검증 세트 손실: 1.098785	최선의 손실: 1.082460	정확도: 62.67%
14	검증 세트 손실: 1.072092	최선의 손실: 1.072092	정확도: 58.00%
15	검증 세트 손실: 1.160124	최선의 손실: 1.072092	정확도: 57.33%
16	검증 세트 손실: 1.191662	최선의 손실: 1.072092	정확도: 53.33%
17	검증 세트 손실: 1.094844	최선의 손실: 1.072092	정확도: 64.00%
18	검증 세트 손실: 1.103728	

이제 이전과 거의 동일한 코드로m 모델을 훈련합니다. 다른 점은 시작할 때 (훈련 세트와 검증 세트 모두) 동결층의 맨 꼭대기 층의 출력을 계산해서 캐싱하는 것입니다. 이렇게 하면 거의 1.5에서 3배 정도 훈련이 빨라집니다(시스템에 따라 이 수치는 매우 달라집니다)

### d.

In [43]:
tf.reset_default_graph()

restore_saver = tf.train.import_meta_graph("models/chap11_exercise.ckpt.meta")

X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")

hidden4_out = tf.get_default_graph().get_tensor_by_name("dnn/hidden4/Elu:0")
logits = tf.layers.dense(hidden4_out, n_outputs, kernel_initializer=he_init, name="new_logits")
Y_proba = tf.nn.softmax(logits)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

learning_rate = 0.01

output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="new_logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam2")
training_op = optimizer.minimize(loss, var_list=output_layer_vars)

init = tf.global_variables_initializer()
four_frozen_saver = tf.train.Saver()

In [45]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "models/chap11_exercise.ckpt")
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = four_frozen_saver.save(sess, "models/chap11_exercise_four.ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("조기 종료!")
                break
        print("{}\t검증 세트 손실: {:.6f}\t최선의 손실: {:.6f}\t정확도: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    four_frozen_saver.restore(sess, "models/chap11_exercise_four.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("최종 테스트 정확도: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from models/chap11_exercise.ckpt
0	검증 세트 손실: 1.381814	최선의 손실: 1.381814	정확도: 49.33%
1	검증 세트 손실: 1.081485	최선의 손실: 1.081485	정확도: 60.67%
2	검증 세트 손실: 1.198649	최선의 손실: 1.081485	정확도: 48.67%
3	검증 세트 손실: 1.149890	최선의 손실: 1.081485	정확도: 57.33%
4	검증 세트 손실: 1.088721	최선의 손실: 1.081485	정확도: 56.00%
5	검증 세트 손실: 1.033008	최선의 손실: 1.033008	정확도: 62.67%
6	검증 세트 손실: 1.051053	최선의 손실: 1.033008	정확도: 62.67%
7	검증 세트 손실: 1.106173	최선의 손실: 1.033008	정확도: 56.67%
8	검증 세트 손실: 1.070067	최선의 손실: 1.033008	정확도: 56.67%
9	검증 세트 손실: 1.077142	최선의 손실: 1.033008	정확도: 61.33%
10	검증 세트 손실: 1.016346	최선의 손실: 1.016346	정확도: 68.00%
11	검증 세트 손실: 0.991142	최선의 손실: 0.991142	정확도: 64.00%
12	검증 세트 손실: 0.931112	최선의 손실: 0.931112	정확도: 70.67%
13	검증 세트 손실: 1.032602	최선의 손실: 0.931112	정확도: 63.33%
14	검증 세트 손실: 1.051768	최선의 손실: 0.931112	정확도: 63.33%
15	검증 세트 손실: 0.980843	최선의 손실: 0.931112	정확도: 67.33%
16	검증 세트 손실: 0.941269	최선의 손실: 0.931112	정확도: 64.00%
17	검증 세트 손실: 1.113663	최선의 손실: 0.931112	정확도: 58.67%
18	검증 세트 손실: 1.010767	

아주 훌륭하지는 않지만 더 나아졌다.

### e.

In [47]:
learning_rate = 0.01

unfrozen_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="hidden[34]|new_logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam3")
training_op = optimizer.minimize(loss, var_list=unfrozen_vars)

init = tf.global_variables_initializer()
two_frozen_saver = tf.train.Saver()

In [49]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    four_frozen_saver.restore(sess, "models/chap11_exercise_four.ckpt")
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = two_frozen_saver.save(sess, "models/chap11_exercise_two_frozen.ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("조기 종료!")
                break
        print("{}\t검증 세트 손실: {:.6f}\t최선의 손실: {:.6f}\t정확도: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    two_frozen_saver.restore(sess, "models/chap11_exercise_two_frozen.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("최종 테스트 정확도: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from models/chap11_exercise_four.ckpt
0	검증 세트 손실: 1.560328	최선의 손실: 1.560328	정확도: 62.00%
1	검증 세트 손실: 1.122247	최선의 손실: 1.122247	정확도: 68.00%
2	검증 세트 손실: 0.939205	최선의 손실: 0.939205	정확도: 70.67%
3	검증 세트 손실: 1.047074	최선의 손실: 0.939205	정확도: 73.33%
4	검증 세트 손실: 0.942020	최선의 손실: 0.939205	정확도: 76.00%
5	검증 세트 손실: 0.827583	최선의 손실: 0.827583	정확도: 78.00%
6	검증 세트 손실: 0.896122	최선의 손실: 0.827583	정확도: 76.00%
7	검증 세트 손실: 1.104361	최선의 손실: 0.827583	정확도: 78.00%
8	검증 세트 손실: 1.336321	최선의 손실: 0.827583	정확도: 73.33%
9	검증 세트 손실: 0.991482	최선의 손실: 0.827583	정확도: 76.00%
10	검증 세트 손실: 1.123702	최선의 손실: 0.827583	정확도: 80.67%
11	검증 세트 손실: 1.009997	최선의 손실: 0.827583	정확도: 78.67%
12	검증 세트 손실: 0.994052	최선의 손실: 0.827583	정확도: 77.33%
13	검증 세트 손실: 1.200181	최선의 손실: 0.827583	정확도: 75.33%
14	검증 세트 손실: 1.038967	최선의 손실: 0.827583	정확도: 75.33%
15	검증 세트 손실: 1.067715	최선의 손실: 0.827583	정확도: 78.67%
16	검증 세트 손실: 1.306437	최선의 손실: 0.827583	정확도: 79.33%
17	검증 세트 손실: 1.115694	최선의 손실: 0.827583	정확도: 77.33%
18	검증 세트 손실: 1.63

성능이 그렇게 좋아지지는 않았다. 모든 층을 동결 해제하고 정확도를 계산해 보자.

In [50]:
learning_rate = 0.01

optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam_4")
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
zero_frozen_saver = tf.train.Saver()

In [54]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    two_frozen_saver.restore(sess, "models/chap11_exercise_two_frozen.ckpt")
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = zero_frozen_saver.save(sess, "models/chap11_exercise_two_frozen.ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("조기 종료!")
                break
        print("{}\t검증 세트 손실: {:.6f}\t최선의 손실: {:.6f}\t정확도: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    zero_frozen_saver.restore(sess, "models/chap11_exercise_two_frozen.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("최종 테스트 정확도: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from models/chap11_exercise_two_frozen.ckpt
0	검증 세트 손실: 1.092992	최선의 손실: 1.092992	정확도: 82.00%
1	검증 세트 손실: 1.448133	최선의 손실: 1.092992	정확도: 80.00%
2	검증 세트 손실: 0.893574	최선의 손실: 0.893574	정확도: 90.67%
3	검증 세트 손실: 0.725983	최선의 손실: 0.725983	정확도: 91.33%
4	검증 세트 손실: 0.911656	최선의 손실: 0.725983	정확도: 88.67%
5	검증 세트 손실: 0.588267	최선의 손실: 0.588267	정확도: 92.00%
6	검증 세트 손실: 0.879624	최선의 손실: 0.588267	정확도: 91.33%
7	검증 세트 손실: 1.328020	최선의 손실: 0.588267	정확도: 86.00%
8	검증 세트 손실: 1.250800	최선의 손실: 0.588267	정확도: 88.67%
9	검증 세트 손실: 1.453489	최선의 손실: 0.588267	정확도: 92.00%
10	검증 세트 손실: 1.497060	최선의 손실: 0.588267	정확도: 91.33%
11	검증 세트 손실: 1.642997	최선의 손실: 0.588267	정확도: 91.33%
12	검증 세트 손실: 1.682036	최선의 손실: 0.588267	정확도: 91.33%
13	검증 세트 손실: 1.712296	최선의 손실: 0.588267	정확도: 91.33%
14	검증 세트 손실: 1.726808	최선의 손실: 0.588267	정확도: 91.33%
15	검증 세트 손실: 1.734523	최선의 손실: 0.588267	정확도: 91.33%
16	검증 세트 손실: 1.738936	최선의 손실: 0.588267	정확도: 91.33%
17	검증 세트 손실: 1.742714	최선의 손실: 0.588267	정확도: 91.33%
18	검증 세트 손실

처음부터 훈련시킨 DNN과 비교해보자.

In [58]:
dnn_clf_5_to_9 = DNNClassifier(n_hidden_layers=4)
dnn_clf_5_to_9.fit(X_train2, y_train2, n_epochs=1000, X_valid=X_valid2, y_valid=y_valid2)

0	검증 세트 손실: 0.472236	최선의 손실: 0.472236	정확도: 88.00%
1	검증 세트 손실: 0.657322	최선의 손실: 0.472236	정확도: 84.00%
2	검증 세트 손실: 0.555275	최선의 손실: 0.472236	정확도: 83.33%
3	검증 세트 손실: 1.002466	최선의 손실: 0.472236	정확도: 82.67%
4	검증 세트 손실: 0.618466	최선의 손실: 0.472236	정확도: 89.33%
5	검증 세트 손실: 0.724875	최선의 손실: 0.472236	정확도: 90.67%
6	검증 세트 손실: 0.924811	최선의 손실: 0.472236	정확도: 87.33%
7	검증 세트 손실: 1.568413	최선의 손실: 0.472236	정확도: 83.33%
8	검증 세트 손실: 2.048452	최선의 손실: 0.472236	정확도: 71.33%
9	검증 세트 손실: 0.855954	최선의 손실: 0.472236	정확도: 90.00%
10	검증 세트 손실: 0.706350	최선의 손실: 0.472236	정확도: 83.33%
11	검증 세트 손실: 3.180080	최선의 손실: 0.472236	정확도: 79.33%
12	검증 세트 손실: 0.630881	최선의 손실: 0.472236	정확도: 89.33%
13	검증 세트 손실: 1.021825	최선의 손실: 0.472236	정확도: 91.33%
14	검증 세트 손실: 1.657358	최선의 손실: 0.472236	정확도: 82.00%
15	검증 세트 손실: 0.927330	최선의 손실: 0.472236	정확도: 91.33%
16	검증 세트 손실: 0.989501	최선의 손실: 0.472236	정확도: 92.00%
17	검증 세트 손실: 1.000915	최선의 손실: 0.472236	정확도: 92.00%
18	검증 세트 손실: 1.016654	최선의 손실: 0.472236	정확도: 92.00%
19	검증 세트 손실: 1.018881	최선의 손실: 0.472236	정확

C:\Users\ud803\Anaconda3\envs\hands_on_ml\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


DNNClassifier(activation=<function elu at 0x00000218F707DF78>,
              batch_norm_momentum=None, batch_size=20, dropout_rate=None,
              initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x00000218820A9F48>,
              learning_rate=0.01, n_hidden_layers=4, n_neurons=100,
              optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
              random_staet=None)

In [60]:
from sklearn.metrics import accuracy_score

y_pred = dnn_clf_5_to_9.predict(X_test2)
accuracy_score(y_test2, y_pred)

0.8559967084961942

그래도 어느정도 발전은 있다!